# 데이터처리

- 데이터 잘못된 이미지 삭제

In [ ]:
import pandas as pd

jeju_poi = pd.read_excel('./data/220118/_제주도_POI_have_image.xlsx', index_col=0)

jeju_poi.head(1)

,keyword,address_name,category_group_code,category_group_name,category_name,distance,id,phone,place_name,place_url,road_address_name,x,y,rating,img_link
0,테마파크,제주특별자치도 제주시 연동 1320,CT1,관광명소,"문화,예술 > 문화시설 > 박물관",NaN,26388484,064-742-3700,수목원테마파크 아이스뮤지엄,http://place.map.kakao.com/26388484,제주특별자치도 제주시 은수길 69,126.488398,33.470777,1.0,https://img1.kakaocdn.net/relay/local/R640x320...


## 1. 사용할 이미지 확인
- 로고, 장소와 다른 이미지를 삭제하고 남은 이미지 확인
  - 파일명 : (idx)_(plac ename).jpg
- .jpg를 삭제하고 '_'를 기준으로 split

In [ ]:
import os

path = './data/220118/_제주도_POI_IMAGE/'
use_idx = []
for dir_name in os.listdir(path):
    print(dir_name)
    for image in os.listdir(path + dir_name):
        image = image.replace('.jpg', '')
        idx, p_name = image.split('_')
        use_idx.append(int(idx))

관광명소
음식점
카페


In [ ]:
use_idx[:10], len(use_idx)

([0, 1031, 1032, 1035, 1036, 1037, 1039, 1045, 1046, 1047], 2306)

## 2. 남은 이미지만 사용
- use_idx를 이용하여 해당하는 인덱스만 사용

In [ ]:
use_idx.sort()

In [ ]:
jeju_use = jeju_poi.loc[use_idx, :].copy()

In [ ]:
jeju_use.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2306 entries, 0 to 5740
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   keyword              2306 non-null   object 
 1   address_name         2306 non-null   object 
 2   category_group_code  2306 non-null   object 
 3   category_group_name  2306 non-null   object 
 4   category_name        2306 non-null   object 
 5   distance             0 non-null      float64
 6   id                   2306 non-null   int64  
 7   phone                1588 non-null   object 
 8   place_name           2306 non-null   object 
 9   place_url            2306 non-null   object 
 10  road_address_name    1989 non-null   object 
 11  x                    2306 non-null   float64
 12  y                    2306 non-null   float64
 13  rating               2306 non-null   float64
 14  img_link             2306 non-null   object 
dtypes: float64(4), int64(1), object(10)
me

In [ ]:
jeju_use.head(1)

,keyword,address_name,category_group_code,category_group_name,category_name,distance,id,phone,place_name,place_url,road_address_name,x,y,rating,img_link
0,테마파크,제주특별자치도 제주시 연동 1320,CT1,관광명소,"문화,예술 > 문화시설 > 박물관",NaN,26388484,064-742-3700,수목원테마파크 아이스뮤지엄,http://place.map.kakao.com/26388484,제주특별자치도 제주시 은수길 69,126.488398,33.470777,1.0,https://img1.kakaocdn.net/relay/local/R640x320...


In [ ]:
# 데이터 저장
jeju_use.to_excel('./data/220118/_제주도_POI_final_use.xlsx')

## 3. 이미지 RESIZE
- 이미지의 크기가 다르기 때문에 동일한 크기로 조절
  - 사이즈 : (224, 224)

### 3-(1). 이미지 Path 확인
- 폴더별로 이미지가 나누어져 있으므로 폴더를 key 값으로 dict형 구성

In [ ]:
import os

path = './data/220118/_제주도_POI_IMAGE/'
image_path = {'관광명소' : [], '음식점' : [], '카페' : []}
for dir_name in os.listdir(path):
    print(dir_name)
    for image in os.listdir(path + dir_name):
        image_path[dir_name].append(path + dir_name + '/' + image)

관광명소
음식점
카페


In [ ]:
image_path['관광명소'][:2], image_path['음식점'][:2], image_path['카페'][:2]

(['./data/220118/_제주도_POI_IMAGE/관광명소/0_수목원테마파크 아이스뮤지엄.jpg',
  './data/220118/_제주도_POI_IMAGE/관광명소/1031_메이즈랜드.jpg'],
 ['./data/220118/_제주도_POI_IMAGE/음식점/1003_화랑스시.jpg',
  './data/220118/_제주도_POI_IMAGE/음식점/1005_폭폭.jpg'],
 ['./data/220118/_제주도_POI_IMAGE/카페/1014_퍼시몬.jpg',
  './data/220118/_제주도_POI_IMAGE/카페/1020_플로레종.jpg'])

### 3-(2). 이미지 Resize

### ◽PIL 사용
- jpg의 경우 중간에 오류가 발생하여 png로 변경

In [ ]:
from PIL import Image

home = 'C:/Users/Isanghada/Desktop/테스트/'
for kind in image_path.keys():
    for image in image_path[kind]:    
        name = image.split('/')[-1]
        img = Image.open(image)
        img.resize((224, 224)).save(home+kind+'/'+name[:-3]+'png', 'png')

## 4. 이미지 이름 변경
- 제주시, 서귀포시 이미지 합치기
- idx_장소명 -> id_장소명으로 변경``

In [ ]:
# 데이터 읽기
import pandas as pd

poi_df = pd.read_excel('./data/220122/_종합_POI.xlsx', index_col=False)
poi_df['idx'] = poi_df['idx'].astype('str')

poi_df.head(1)

,idx,keyword,address_name,category_group_name,category_name,id,place_name,x,y,rating,kind
0,0,테마파크,제주특별자치도 제주시 연동 1320,관광명소,"문화,예술 > 문화시설 > 박물관",26388484,수목원테마파크 아이스뮤지엄,126.488398,33.470777,1.0,제주시


In [ ]:
poi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3008 entries, 0 to 3007
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   idx                  3008 non-null   object 
 1   keyword              3008 non-null   object 
 2   address_name         3008 non-null   object 
 3   category_group_name  3008 non-null   object 
 4   category_name        3008 non-null   object 
 5   id                   3008 non-null   int64  
 6   place_name           3008 non-null   object 
 7   x                    3008 non-null   float64
 8   y                    3008 non-null   float64
 9   rating               3008 non-null   float64
 10  kind                 3008 non-null   object 
dtypes: float64(3), int64(1), object(7)
memory usage: 258.6+ KB


### 4-(1). 이미지 이름 변경
- 제주시, 서귀포시 => 통합
- shutil을 사용하여 이미지 복사 (idx_장소명 -> id_장소명)

In [ ]:
import os
import shutil

home_path = 'C:/Users/Isanghada/Desktop/이미지/'
kinds = ['제주시', '서귀포시']
move = '통합'
dict_idx = {i : 0 for i in range(6000)}
for kind in kinds:
    print(kind,'------------------')
    for dir_name in os.listdir(home_path+kind):
        print(dir_name)
        for file in os.listdir(home_path+kind+'/'+dir_name):
            p_idx, p_name = file.replace('.png', '').split('_')
            try:
                p_id = poi_df.query(f'idx == "{p_idx}"')['id'].values[dict_idx[int(p_idx)]]
            except :
                print(p_idx, p_name, kind, dir_name)
                continue
            dict_idx[int(p_idx)] += 1
            p_file = str(p_id)+'_'+p_name+'.png'
            shutil.copy(home_path+kind+'/'+dir_name+'/'+file, home_path+move+'/'+dir_name+'/'+p_file)
    print('-----------------------------------')


제주시 ------------------
관광명소
음식점
카페
-----------------------------------
서귀포시 ------------------
관광명소
음식점
카페
-----------------------------------


## 5. 자연어 처리 : 코사인 유사도

- 자연어 처리

In [ ]:
# 1. 데이터 읽기
import pandas as pd

jeju_poi = pd.read_excel('./data/220122/_종합_POI.xlsx', index_col=False)

jeju_poi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3008 entries, 0 to 3007
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   idx                  3008 non-null   int64  
 1   keyword              3008 non-null   object 
 2   address_name         3008 non-null   object 
 3   category_group_name  3008 non-null   object 
 4   category_name        3008 non-null   object 
 5   id                   3008 non-null   int64  
 6   place_name           3008 non-null   object 
 7   x                    3008 non-null   float64
 8   y                    3008 non-null   float64
 9   rating               3008 non-null   float64
 10  kind                 3008 non-null   object 
dtypes: float64(3), int64(2), object(6)
memory usage: 258.6+ KB


In [ ]:
jeju_poi.head(1)

,idx,keyword,address_name,category_group_name,category_name,id,place_name,x,y,rating,kind
0,0,테마파크,제주특별자치도 제주시 연동 1320,관광명소,"문화,예술 > 문화시설 > 박물관",26388484,수목원테마파크 아이스뮤지엄,126.488398,33.470777,1.0,제주시


### 5-(1). 자연어 선택
- keyword와 category_group_name 사용

In [ ]:
keywords = set()
for keyword in jeju_poi['keyword'].unique():
    for k in keyword.split(','):
        keywords.add(k)
keywords = list(keywords)

keywords

['테마파크', '분위기 좋은', '오션뷰', '가족여행', '가볼만한 곳', '체험', '레포츠', '맛집', '감성']

In [ ]:
groups = []

for category in jeju_poi['category_group_name'].unique():
    groups.append(category)

groups

['관광명소', '음식점', '카페']

In [ ]:
jeju_poi[['id', 'place_name', 'keyword', 'category_group_name']].head(1)

,id,place_name,keyword,category_group_name
0,26388484,수목원테마파크 아이스뮤지엄,테마파크,관광명소


### 5-(2). content 만들기
- keyword와 category_group_name를 합쳐 content 컬럼을 추가한다.
  - 자연어를 이용한 유사도 측정을 위해

In [ ]:
from tqdm.notebook import tqdm
jeju_poi['content'] = ''

for idx, row in tqdm(jeju_poi.iterrows()):
    content = (row['keyword']+','+row['category_group_name']).replace(' ', '').replace(',', ' ')
    jeju_poi.loc[idx, 'content'] = content

0it [00:00, ?it/s]

In [ ]:
jeju_poi[['id', 'place_name', 'keyword', 'category_group_name', 'content']].head(2)

,id,place_name,keyword,category_group_name,content
0,26388484,수목원테마파크 아이스뮤지엄,테마파크,관광명소,테마파크 관광명소
1,1129394481,고스트타운,"테마파크,가볼만한 곳",관광명소,테마파크 가볼만한곳 관광명소


In [ ]:
jeju_poi.loc[0, 'content']

'테마파크 관광명소'

In [ ]:
jeju_poi.to_excel('./data/220123/_종합_POI_content.xlsx', index=False)

### 5-(3). CountVectorizer
- 새로 생선한 'content'를 기준으로 CountVectorizer 진행

In [ ]:
import pandas as pd

jeju_poi = pd.read_excel('./data/220123/_종합_POI_content.xlsx', index_col=False)

jeju_poi.head(1)

,idx,keyword,address_name,category_group_name,category_name,id,place_name,x,y,rating,kind,content
0,0,테마파크,제주특별자치도 제주시 연동 1320,관광명소,"문화,예술 > 문화시설 > 박물관",26388484,수목원테마파크 아이스뮤지엄,126.488398,33.470777,1.0,제주시,테마파크 관광명소


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2))
content_mat = count_vect.fit_transform(jeju_poi['content'])
content_mat.shape

(3008, 39)

In [ ]:
count_vect.get_feature_names_out()

array(['가볼만한곳', '가볼만한곳 관광명소', '가족여행', '가족여행 가볼만한곳', '가족여행 관광명소',
       '가족여행 음식점', '감성', '감성 가볼만한곳', '감성 맛집', '감성 음식점', '감성 카페', '관광명소',
       '레포츠', '레포츠 가볼만한곳', '맛집', '맛집 감성', '맛집 분위기좋은', '맛집 오션뷰', '맛집 음식점',
       '맛집 카페', '분위기좋은', '분위기좋은 감성', '분위기좋은 맛집', '분위기좋은 오션뷰', '분위기좋은 음식점',
       '분위기좋은 카페', '오션뷰', '오션뷰 맛집', '오션뷰 음식점', '오션뷰 카페', '음식점', '체험',
       '체험 가볼만한곳', '카페', '테마파크', '테마파크 가볼만한곳', '테마파크 관광명소', '테마파크 레포츠',
       '테마파크 음식점'], dtype=object)

### 5-(4). Cosine Similarity
- content_mat을 활용하여 코사인 유사도 계산

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

content_sim = cosine_similarity(content_mat, content_mat)
print(content_sim.shape)
print(content_sim[:2])

(3008, 3008)
[[1.         0.51639778 0.51639778 ... 0.         0.         0.        ]
 [0.51639778 1.         1.         ... 0.         0.         0.        ]]


- 인덱스를 이용해 원하는 장소 기준의 cosine similarity 추가하여 사용
  - pandas.DataFrame을 사용하여 정렬 및 순위 획득 가능
- 아래는 예시

In [ ]:
ex_poi = jeju_poi.copy(deep = True)
ex_poi['nl_cosine'] = content_sim[0]

In [ ]:
ex_poi[['id', 'place_name', 'nl_cosine']].head(5)

,id,place_name,nl_cosine
0,26388484,수목원테마파크 아이스뮤지엄,1.000000
1,1129394481,고스트타운,0.516398
2,891104398,제주불빛정원,0.516398
3,1868828759,9.81파크,0.516398
4,9401924,제주공룡랜드,0.516398


- 코사인 유사도 array를 저장하여 필요할 때 불러와 사용

In [ ]:
import numpy as np

# np.save('./data/220123/_nl_content_sim.npy', content_sim)

In [ ]:
import numpy as np

np_file = np.load(file='./data/220123/_nl_content_sim.npy')
np_file[0][:5]

array([1.        , 0.51639778, 0.51639778, 0.51639778, 0.51639778])